In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from habanero import Crossref

In [ ]:
base_path = 'source/'
use_column = ['Article Title', 'Abstract', 'DOI']

In [ ]:
def excel_list():
    folder = Path(base_path)
    return [f.name for f in folder.iterdir() if f.is_file()]

#e = excel_list()

In [ ]:
def make_one(excel_list):

    concat_list = []

    def cutted_df(file_name):
        d = pd.read_excel(base_path + file_name)
        d = d[use_column]
        d = d.dropna(subset = use_column)
        return d
    
    for e in excel_list:
        concat_list.append(cutted_df(e))

    result = pd.concat(concat_list, axis=0, ignore_index=True)
    return result.drop_duplicates(subset=['DOI'])

#m = make_one(e)
#m.to_csv('make_one.csv', index=False)

In [ ]:
#import date_multi

#dt = pd.read_csv('make_one.csv')
#dt = date_multi.parallel_fetch(dt)
#dt.to_csv('date_one.csv', index=False)